In [27]:
import pandas as pd
import numpy as np

# read in the Excel file
#xlsx = pd.ExcelFile('data_files/llp.xlsx')
xlsx = pd.ExcelFile('llp.xlsx')

# dictionary to store the dataframes from all excel sheets
dfs = {}

for sheet_name in xlsx.sheet_names:
    dfs[sheet_name] = xlsx.parse(sheet_name)

data_frames = dfs.copy()  # for saving initial data frames

In [ ]:
"""
Some of sheets contains 'RSP-EUR' or 'LLP-EUR' or both.  
Depending on this fact it is necessary to prepare dataframes with necessary columns only. 
"Model name", "Characteristic" and "Price" which can be 'RSP-EUR' or 'LLP-EUR' or both. 
To exclude lines with options for product models, it is necessary to exclude line where 
"Characteristic" not empty. 
All prepared dataframes are added to a list which will be used for dataframes merging. 
""" ""

dfs = data_frames.copy()

rsp = 'RSP-EUR'
llp = 'LLP-EUR'

l_for_merging = []
for key in dfs.keys():
    # keys - names of sheets in excel and they have explicit name with time line identification
    # they are used for naming columns with price
    if rsp in dfs[key].columns and llp in dfs[key].columns:
        dfs[key] = dfs[key].loc[:, ['Model Number', 'Characteristic', 'RSP-EUR', 'LLP-EUR']]
        dfs[key] = dfs[key][dfs[key]['Characteristic'].isna()]
        dfs[key] = dfs[key].loc[:, ['Model Number', 'RSP-EUR', 'LLP-EUR']]
        dfs[key] = dfs[key].rename(columns={'RSP-EUR': key + '_' + 'RSP-EUR', 'LLP-EUR': key + '_' + 'LLP-EUR'})
        l_for_merging.append(dfs[key])
    elif rsp in dfs[key].columns:
        dfs[key] = dfs[key].loc[:, ['Model Number', 'Characteristic', 'RSP-EUR']]
        dfs[key] = dfs[key][dfs[key]['Characteristic'].isna()]
        dfs[key] = dfs[key].loc[:, ['Model Number', 'RSP-EUR']]
        dfs[key] = dfs[key].rename(columns={'RSP-EUR': key + '_' + 'RSP-EUR'})
        l_for_merging.append(dfs[key])
    elif llp in dfs[key].columns:
        dfs[key] = dfs[key].loc[:, ['Model Number', 'Characteristic', 'LLP-EUR']]
        dfs[key] = dfs[key][dfs[key]['Characteristic'].isna()]
        dfs[key] = dfs[key].loc[:, ['Model Number', 'LLP-EUR']]
        dfs[key] = dfs[key].rename(columns={'LLP-EUR': key + '_' + 'LLP-EUR'})
        l_for_merging.append(dfs[key])
        
# determination of the dataframe with the biggest quantity of product models
max_l = 0
for i, df in enumerate(l_for_merging):
    print(len(l_for_merging))
    if len(l_for_merging) >= max_l:
        max_l = len(l_for_merging)
        max_df = df
        max_i = i
# creation a dataframe with only one column "Model Number" for left-merging for keeping all models
model_df = l_for_merging[max_i].loc[:, ['Model Number']]        
        
# creation of final dataframe with all prices columns for all products models
final_df = model_df.copy()
for df in l_for_merging:
    final_df = final_df.merge(df, how='left', left_on='Model Number', right_on='Model Number')        

In [35]:
# it is necessary to understand price different in % for 23 and 24 years for LLP and RSP prices
final_df['difference LLP - v4 / v3'] = round((final_df['LLPv4_LLP-EUR'] / final_df['LLPv3_LLP-EUR'] * 100 - 100), 0)
final_df['difference RSP - v4 / v3'] = round((final_df['RSPv4_RSP-EUR'] / final_df['RSPv3_RSP-EUR'] * 100 - 100), 0)
final_df['difference RSP'] = round((final_df['RSPv4_RSP-EUR'] / final_df['RSP23_RSP-EUR'] * 100 - 100), 0)
final_df['difference LLP'] = round((final_df['LLPv4_LLP-EUR'] / final_df['LLP23_LLP-EUR'] * 100 - 100), 0)

In [ ]:
writer = pd.ExcelWriter('data_files/pricebookcheck.xlsx')
final_df.to_excel(writer, sheet_name='summary')
writer.close()
print('You file is ready')